<a href="https://colab.research.google.com/github/Nayasha24/Admission-management-system/blob/main/Text_Classification_Attention_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d matleonard/nlp-course

 99% 57.0M/57.8M [00:02<00:00, 37.0MB/s]
100% 57.8M/57.8M [00:02<00:00, 25.9MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/nlp-course.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Attention, Concatenate

In [5]:
data = pd.read_csv('spam.csv')

In [6]:
texts = data['text'].values
labels = data['label'].values

In [7]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [11]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_seq, maxlen=max_len)
x_test_pad = pad_sequences(x_test_seq, maxlen=max_len)

In [12]:
embedding_dim = 50
lstm_units = 50

In [17]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(input_layer)
bi_lstm_layer = Bidirectional(LSTM(lstm_units, return_sequences=True))(embedding_layer)
attention = Attention()([bi_lstm_layer, bi_lstm_layer])
context = Concatenate(axis=-1)([bi_lstm_layer, attention])
output_layer = Dense(1, activation='sigmoid')(context[:, -1, :])



In [18]:
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [19]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=32, validation_data=(x_test_pad, y_test))

Epoch 1/5
140/140 [==============================] - 25s 137ms/step - loss: 0.2039 - accuracy: 0.9302 - val_loss: 0.0669 - val_accuracy: 0.9839
Epoch 2/5
140/140 [==============================] - 20s 141ms/step - loss: 0.0292 - accuracy: 0.9919 - val_loss: 0.0607 - val_accuracy: 0.9830
Epoch 3/5
140/140 [==============================] - 18s 129ms/step - loss: 0.0102 - accuracy: 0.9975 - val_loss: 0.0531 - val_accuracy: 0.9848
Epoch 4/5
140/140 [==============================] - 20s 142ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 0.0664 - val_accuracy: 0.9857
Epoch 5/5
140/140 [==============================] - 18s 130ms/step - loss: 5.1916e-04 - accuracy: 1.0000 - val_loss: 0.0806 - val_accuracy: 0.9848
